## Model I/O

In [2]:
from dotenv import load_dotenv
load_dotenv()

True

### Prompts(프롬프트)

모델에게 던지는 질문을 템플릿화 하는 기술

**주요 사용처**

1. **자동화된 입력 구성**
    - PromptTemplate을 사용하여 사용자 입력을 자동으로 구성.
    - 동일한 형식의 질문이나 대화를 대량으로 생성 가능

2. **대화형 응답**
    - ChatPromptTemplate을 통해 대화형 AI의 문맥 유지를 지원.

3. **결과 파싱**
    - Output Parsers 를 통해 LLM의 출력을 특정 포맷으로 처리하여 후속 작업 자동화.

In [3]:
%pip install langchain-core

Note: you may need to restart the kernel to use updated packages.


In [4]:
# PromptTemplate
from langchain_core.prompts import PromptTemplate

# 템플릿 정의
template = "{product}를 홍보하기 위한 재미있고, 새로운 광고문구를 작성해 주세요."

# PromptTempalate 객체 생성
prompt = PromptTemplate(
    template=template,
    input_variables=['product']
)

# 실제 프롬프트 생성 (변수 채우기)
print(prompt.format(product='카메라'))
print(prompt.format(product='자동차'))

c:\Users\Admin\miniconda3\envs\pystudy_env\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


카메라를 홍보하기 위한 재미있고, 새로운 광고문구를 작성해 주세요.
자동차를 홍보하기 위한 재미있고, 새로운 광고문구를 작성해 주세요.


In [5]:
# ChatPromptTemplate - System, Human, AI 유형별 메시지를 작성할수 있는 기능 제공
from langchain_core.prompts.chat import ChatPromptTemplate, SystemMessagePromptTemplate, HumanMessagePromptTemplate

# 시스템 메세지 (역할 부여)
system_message = SystemMessagePromptTemplate.from_template("당신은 도움을 주는 친절한 챗봇입니다.")

# 휴먼 메세지 (사용자 질문)
human_message = HumanMessagePromptTemplate.from_template("질문 : {question}")

# 챗 프롬프트 합치기
messages = ChatPromptTemplate.from_messages([system_message, human_message])
print(messages)

# 포맷팅 확인
prompt = messages.format_messages(question="AI가 무엇인가요?")
print(prompt)

input_variables=['question'] input_types={} partial_variables={} messages=[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], input_types={}, partial_variables={}, template='당신은 도움을 주는 친절한 챗봇입니다.'), additional_kwargs={}), HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['question'], input_types={}, partial_variables={}, template='질문 : {question}'), additional_kwargs={})]
[SystemMessage(content='당신은 도움을 주는 친절한 챗봇입니다.', additional_kwargs={}, response_metadata={}), HumanMessage(content='질문 : AI가 무엇인가요?', additional_kwargs={}, response_metadata={})]


In [6]:
# FewShotPromptTemplate - n-shot 프롬프팅을 위한 클래스 (말보다 '예시'로 가르치기)
from langchain_core.prompts.few_shot import FewShotPromptTemplate

# 예시 데이터
examples = [
    {"question": "2 + 2는 무엇인가요?", "answer" : "2 + 2 = 4"},
    {"question": "3 + 5는 무엇인가요?", "answer" : "3 + 5 = 8"},
]

# 예시를 어떻게 보여줄지 포맷 정의
example_prompt = PromptTemplate(
    template="Q: {question}\nA: {answer}",
    input_variables=['question', 'answer']
)

# FewShotPromptTemplate 조립
few_shot_prompt = FewShotPromptTemplate(
    examples=examples,                   # 예시 데이터
    example_prompt=example_prompt,       # 예시 포맷
    prefix="다음 계산문제를 해결하세요",     # 지시어 (앞부분)
    suffix="Q: {question}\nA:",          # 실제 질문 (뒷부분)
    input_variables=['question']
)

prompt = few_shot_prompt.format(question="35 + 48은 무엇인가요?")
print(prompt)

다음 계산문제를 해결하세요

Q: 2 + 2는 무엇인가요?
A: 2 + 2 = 4

Q: 3 + 5는 무엇인가요?
A: 3 + 5 = 8

Q: 35 + 48은 무엇인가요?
A:


### Output Parser(출력 파서)

LLM이 생성한 텍스트 출력을 특정 형식으로 변환하거나 처리하는 데 사용
- 텍스트 기반 응답을 JSON, 리스트 또는 숫자와 같은 특정 포맷으로 변환하여 후속 작업에 활용

**종류**
1. **BaseOutputParser**: Output Parsers의 기본 클래스, 커스텀 파서 구현 시 사용
2. **CommaSeparatedListOutputParser**: 콤마로 구분된 문자열을 리스트로 변환
3. **RegexParser**: 정규식을 사용해 특정 패턴을 추출하고 키-값 형태로 반환
4. **StructuredOutputParser**: 출력의 JSON 또는 구조화된 형식을 강제
5. **PydanticOutputParser**: Pydantic 모델을 기반으로 출력 검증 및 변환
6. **MarkdownOutputParser**: 마크다운 형식의 텍스트에서 데이터를 추출

In [7]:
%pip install langchain-openai

Note: you may need to restart the kernel to use updated packages.


In [8]:
from langchain_core.output_parsers import CommaSeparatedListOutputParser
from langchain_openai import ChatOpenAI

# 파서 생성
output_parser = CommaSeparatedListOutputParser()

# 지시사항 가져오기
format_instructions = output_parser.get_format_instructions()
# print(format_instructions)

# 프롬프트 템플릿에 지시사항 주입
prompt_template = PromptTemplate(
    template="{subject} 5개의 팀을 보여주세요.\n형식지정: {format_instructions}",
    input_variables=['subject'],    # 사용자 입력값 변수
    partial_variables={'format_instructions' : format_instructions} # 고정적(기본적으로 포함)으로 설정될 변수
)

# 모델 준비
model = ChatOpenAI(model_name='gpt-5-nano', temperature=0)

query = "한국의 프로야구팀"
prompt = prompt_template.format(subject=query)

# invoke로 실행
response = model.invoke(prompt)
print("원본 응답:", response.content)

# 파싱 (문자열 -> 리스트로 변환)
parsed_result = output_parser.parse(response.content)
print("파싱 결과:", parsed_result)

원본 응답: 두산 베어스, 한화 이글스, 기아 타이거즈, 키움 히어로즈, LG 트윈스
파싱 결과: ['두산 베어스', '한화 이글스', '기아 타이거즈', '키움 히어로즈', 'LG 트윈스']


### Models(다양한 모델 사용하기)

In [9]:
# ChatOpneAI

from langchain_openai import ChatOpenAI

model_gpt = ChatOpenAI(model="gpt-5-nano", temperature=1)
model_gpt.invoke("저는 아침으로 사과를 먹었습니다. 저는 아침에 무엇을 먹었을까요?")

AIMessage(content='네, 아침으로 사과를 먹으셨어요.  \n원하시면 사과와 함께 먹기 좋은 간단한 아침 아이디어도 알려드릴게요.', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 560, 'prompt_tokens': 28, 'total_tokens': 588, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 512, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_provider': 'openai', 'model_name': 'gpt-5-nano-2025-08-07', 'system_fingerprint': None, 'id': 'chatcmpl-Cqq4Nu5ANc31WwGPsT8B74xABfG0n', 'service_tier': 'default', 'finish_reason': 'stop', 'logprobs': None}, id='lc_run--019b5813-d681-71d3-a8f8-67c1294ca32f-0', usage_metadata={'input_tokens': 28, 'output_tokens': 560, 'total_tokens': 588, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 512}})

In [10]:
%pip install langchain-huggingface

Note: you may need to restart the kernel to use updated packages.


In [11]:
# HuggingFaceEndpoint(오픈소스 모델)
from langchain_huggingface import HuggingFaceEndpoint, ChatHuggingFace

# 엔드포인트 설정
end_point = HuggingFaceEndpoint(
    repo_id='MLP-KTLim/llama-3-Korean-Bllossom-8B',
    task='text-generation',
    max_new_tokens=1024
)

# 챗 모델로 감싸기
hf_model = ChatHuggingFace(llm=end_point)

hf_model.invoke("저는 아침으로 사과를 먹었습니다. 저는 아침에 무엇을 먹었을까요?")

AIMessage(content='잘했어요! 아침에 사과를 먹었어요!', additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 13, 'prompt_tokens': 43, 'total_tokens': 56}, 'model_name': 'MLP-KTLim/llama-3-Korean-Bllossom-8B', 'system_fingerprint': '', 'finish_reason': 'stop', 'logprobs': None}, id='lc_run--019b5814-0ba2-70f0-b24c-086eb12408d4-0', usage_metadata={'input_tokens': 43, 'output_tokens': 13, 'total_tokens': 56})

In [12]:
%pip install -U huggingface_hub

  Using cached huggingface_hub-1.2.3-py3-none-any.whl.metadata (13 kB)
   ---------------------------------------- 0.0/2.9 MB ? eta -:--:--
   -------------- ------------------------- 1.0/2.9 MB 8.4 MB/s eta 0:00:01
   ------------------------- -------------- 1.8/2.9 MB 4.2 MB/s eta 0:00:01
   ------------------------------------ --- 2.6/2.9 MB 4.1 MB/s eta 0:00:01
   ---------------------------------------- 2.9/2.9 MB 4.0 MB/s  0:00:00

   ---------- ----------------------------- 1/4 [hf-xet]
  Attempting uninstall: huggingface_hub
   ---------- ----------------------------- 1/4 [hf-xet]
    Found existing installation: huggingface-hub 0.36.0
   ---------- ----------------------------- 1/4 [hf-xet]
   ------------------------------ --------- 3/4 [huggingface_hub]
    Uninstalling huggingface-hub-0.36.0:
   ------------------------------ --------- 3/4 [huggingface_hub]
      Successfully uninstalled huggingface-hub-0.36.0
   ------------------------------ --------- 3/4 [huggingface_hub

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
langchain-huggingface 1.2.0 requires huggingface-hub<1.0.0,>=0.33.4, but you have huggingface-hub 1.2.3 which is incompatible.
transformers 4.57.3 requires huggingface-hub<1.0,>=0.34.0, but you have huggingface-hub 1.2.3 which is incompatible.


### ModelLaboratory(모델비교)

In [15]:
%pip install langchain-classic

  Using cached langchain_text_splitters-1.1.0-py3-none-any.whl.metadata (2.7 kB)
  Using cached sqlalchemy-2.0.45-cp312-cp312-win_amd64.whl.metadata (9.8 kB)
  Using cached greenlet-3.3.0-cp312-cp312-win_amd64.whl.metadata (4.2 kB)
   ---------------------------------------- 0.0/1.0 MB ? eta -:--:--
   ---------------------------------------- 1.0/1.0 MB 16.6 MB/s  0:00:00
Using cached langchain_text_splitters-1.1.0-py3-none-any.whl (34 kB)
Using cached sqlalchemy-2.0.45-cp312-cp312-win_amd64.whl (2.1 MB)
Using cached greenlet-3.3.0-cp312-cp312-win_amd64.whl (301 kB)

   ---------------------------------------- 0/4 [greenlet]
   ---------- ----------------------------- 1/4 [sqlalchemy]
   ---------- ----------------------------- 1/4 [sqlalchemy]
   ---------- ----------------------------- 1/4 [sqlalchemy]
   ---------- ----------------------------- 1/4 [sqlalchemy]
   ---------- ----------------------------- 1/4 [sqlalchemy]
   ---------- ----------------------------- 1/4 [sqlalchemy]
 

In [16]:
from langchain_classic.model_laboratory import ModelLaboratory

model_lab = ModelLaboratory.from_llms([model_gpt, hf_model])
model_lab.compare("대한민국의 겨울은 몇 월부터 몇 월인가요??")

Input:
대한민국의 겨울은 몇 월부터 몇 월인가요??

profile={'max_input_tokens': 272000, 'max_output_tokens': 128000, 'image_inputs': True, 'audio_inputs': False, 'video_inputs': False, 'image_outputs': False, 'audio_outputs': False, 'video_outputs': False, 'reasoning_output': True, 'tool_calling': True, 'structured_output': True, 'image_url_inputs': True, 'pdf_inputs': True, 'pdf_tool_message': True, 'image_tool_message': True, 'tool_choice': True} client=<openai.resources.chat.completions.completions.Completions object at 0x00000153326CB650> async_client=<openai.resources.chat.completions.completions.AsyncCompletions object at 0x00000153326CA420> root_client=<openai.OpenAI object at 0x00000153326CA630> root_async_client=<openai.AsyncOpenAI object at 0x00000153326CA7B0> model_name='gpt-5-nano' temperature=1.0 model_kwargs={} openai_api_key=SecretStr('**********') stream_usage=True
일반적으로 두 가지 정의가 있어요.

- 기상학적 겨울: 12월, 1월, 2월. 한국의 기상청 등에서 사용하는 표준 정의입니다.
- 천문학적 겨울: 겨울분점(동지) 시점부터 춘분 시점까지로, 대략 12월 말경부터 3월 초경